# Tube à bulles : validation sur le cas Hibiki & Ishii

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
matplotlib.rcParams['text.usetex'] = True
import copy
import os, sys
from string import Template
import time
from IPython.display import display
import math
from functions import *

from trustutils import run 
from trustutils import visit
from trustutils.jupyter import plot

run.TRUST_parameters()

## Préparation des calculs
A remplir par l'utilisateur

In [ ]:
#Create the calculation cases
force_recalculation = True
build = run.BUILD_DIRECTORY # build directory (absolute path !)
number_of_partitions = 10

# experimental data
config_exp = "Hibiki"
carac_essai = pd.read_csv(f"donnees/{config_exp}.csv")
n_exp = len(carac_essai["SetNumber"])

# Numerical parameters
facsec  = [ 1, 1] # mesh
tmax    = 20
nb_pas_dt_max = '100000'
seuil_statio  = '1.e-3'
solveur = 'ice'

param_sch = [tmax, facsec, nb_pas_dt_max, seuil_statio, solveur]

Récupérer les conditions expérimentales nécessaire au lancement des simulation

In [ ]:
# Retrieve experiment data
carac_essai["alpha_l0"] = np.array(carac_essai['JF'])/(np.array(carac_essai['JF'])+np.array(carac_essai['JG']))
carac_essai["alpha_g0"] = np.array(carac_essai['JG'])/(np.array(carac_essai['JF'])+np.array(carac_essai['JG']))
carac_essai["u_0"]      = np.array(carac_essai['JF'])/np.array(carac_essai["alpha_l0"])
carac_essai["CL_k"]     = 0.01*np.array(carac_essai["u_0"])**2
carac_essai["CL_om"]    = np.array(carac_essai["u_0"])/np.array(carac_essai["D_h"])
carac_essai["g"]        = [-9.81] * len(carac_essai["SetNumber"])
carac_essai["fluid"]    = ["water-air"] * len(carac_essai["SetNumber"])
carac_essai["mesh"] = ["Hibiki"] * len(carac_essai["SetNumber"])
carac_essai["h"] = [3.061] * len(carac_essai["SetNumber"])
# diamètre bulles
tab_str_diam = ["1.e-3"]*len(carac_essai["SetNumber"]) # Default single phase
for index, row in carac_essai.iterrows():
    expNumber = row["SetNumber"]
    path = f"donnees/HIBIKI/Set{expNumber[1:]}/zD=53.5/exp.csv"
    tab_G = pd.read_csv(path, sep = ',')

    r_loc = np.array(tab_G["r/R"])*row['D_h']/2.
    d_loc = np.array(tab_G["Dav"]) # These are in mm

    tab_str_diam[index] = str_r_loc_d_loc(r_loc, d_loc)

carac_essai["Dbubble"] = tab_str_diam
print(np.array(carac_essai['JF']))

Maillage

In [ ]:
#Create the mesh
#Permission denied to build mesh from jupyter : do this manually...

force_remeshing = False # if False, existing results will be used if the build directory exists, e.g. the trust-nk calculation will be skipped to quickly show results

n_raf   = 1

title_raf  = [ 1, 2 ]
title_exp  = ["Hibiki"] 
n_z    = [ 80, 80] # number of vertical stacks of cells
n_x    = [ 1 ,  2] # number of cells in each quarter of space

## Lancement des calculs

In [ ]:
# Launch calculations

if force_recalculation or not os.path.exists('build'):
    print('Recalculation running..')
    run.reset()
    run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)
    
    # substituion dictionnaries
    dict_mesh_sch = substitution_mesh_sch(n_raf, param_sch) # substitution of mesh and time scheme info
    dict_pb = substitution_pb(carac_essai) # substitution of pb_multiphase info

    
    for r in range(n_raf) :
        print(f"Adding the launch files for a refinement of {title_raf[r]}")
        for index, row in carac_essai.iterrows():
            name_run = f"{config_exp}_{row['SetNumber']}_raf_{title_raf[r]}"
            mesh_run = f"mesh_{row['mesh']}_{title_raf[r]}.med"
            os.system(f'mkdir build/{name_run}')
            os.system(f'cp meshes/{mesh_run} build/{name_run}/{mesh_run}')
            
            GenerateInputFile(f"{build}/{name_run}", build, {**dict_mesh_sch[r], **dict_pb[index]}, "jdd", name_run)
            if (number_of_partitions == 1):
                run.addCase(f"./{build}/{name_run}", f"{name_run}.data")
                print(f"Adding {name_run}.data")
            else :
                os.system(f'cd build/{name_run};  make_PAR.data {name_run} {number_of_partitions}; cd ../..')                            
                run.addCase(f"./{name_run}", f"PAR_{name_run}.data",nbProcs=number_of_partitions)

    run.runCases()
    perf= run.tablePerf()

else:
    print('No recalculation, reusing old results!')

Tableau des performance : décommenter la dernière ligne pour afficher le tableau

In [ ]:
performance = open(build+"/perf.csv", "w")
for r in range(n_raf): # mesh
    for index, row in carac_essai.iterrows(): # experiments
        folder_run = f"/{config_exp}_{row['SetNumber']}_raf_{title_raf[r]}/"
        name_run = f"PAR_{config_exp}_{row['SetNumber']}_raf_{title_raf[r]}"
        performance.write(read_perf(build+folder_run+name_run+".perf", name_run, f"{config_exp}_{row['SetNumber']} raf_{title_raf[r]}"))
performance.close()
performance = open(build+"/perf.csv", "r")
data = pd.read_csv(performance, sep=" ", header=None)
df = data[data.columns[:-1]]
df.columns = ["Configuration", "mesh", "Host", "System", "total_CPU_time", "CPU time/step", "nb_cells"]
blankIndex=[''] * len(df)
df.index=blankIndex
display(df)

Graphique : temps de calcul

In [ ]:
# Graph temps de calcul
grpd = df.groupby('Configuration')
i=1
for name, data in grpd:
    plt.plot(data.nb_cells.values, data.total_CPU_time.values, 'o-', label = f"Liu{i}")
    i += 1
plt.legend(ncol=6,loc='center left', bbox_to_anchor=(1, 0.5), fontsize=12)
plt.xlabel("number of cells")
plt.ylabel("Total CPU time [s]") # diviser par 3600s !!!!!!!!!!!!!!!!!!!!!!!!!!!
plt.title("Calculation time")

## Post traitement
### Evolution des résidus

In [ ]:
## plotting convergence

matplotlib.rcParams.update({'font.size': 14})

color_raf = [ "red", "green", "blue"]

fig = plt.figure(figsize = (16, 4))

name_phys = [ ["vitl", "vitg"], ["alphal", "alphag"],  ["k"], ["diss"]]
par = "PAR_"
if number_of_partitions == 1 : par = ""

axs = fig.subplots(1, 4, sharex=True, sharey=True)

for l in range(len(name_phys)): #phys
    for n, row in carac_essai.iterrows():
        for r in range(n_raf) :

            folder_run = f"{config_exp}_{row['SetNumber']}_raf_{title_raf[r]}"
            name = folder_run

            label_loc = f"{config_exp}_{row['SetNumber']}_raf_{title_raf[r]}"

            residuals = pd.read_csv(f"{build}/{folder_run}/{par}{name}.dt_ev", "\t")
            residuals = residuals.iloc[:,:13]
            residuals.columns = ['time', 'dt', 'facsec', 'residu', 'dt_stab', 'vitl', 'vitg', 'alphal', 'alphag', 'Tl', 'Tg', 'diss', 'k']               

            for lp in range(len(name_phys[l])) : #physique de la phase si multiphase
                axs[l].plot(residuals["time"][::], residuals[name_phys[l][lp]][::], "-", label = label_loc, color = color_raf[r])

    axs[l].set_xlim(0,tmax)
    axs[l].set_ylim(float(seuil_statio)/100, 1e4)
    axs[l].set_yscale("log")
    axs[l].set_title("convergence "+name_phys[l][0] )
    axs[l].set_xlabel('time')
    axs[l].set_ylabel("convergence " + name_phys[l][0])
    #axs[l].legend(prop={'size': 10})

plt.tight_layout()
plt.savefig("convergence.pdf")

### Comparaison des résultats de simulation aux données expérimentales
Stocker les résultats des simulations dans une liste

In [ ]:
# # Recover simulation results in a list
tab_pos = ["", "_DIAG"] # two probes were requested for each simulation
# sim is a table were results are stored for every probe, every mesh and every experiment
sim = [ [ [ -1 for l in range(len(tab_pos)) ] for r in range(n_raf) ] for n in range(n_exp) ]
# every element in sim will be a dataframe (pandas) with results from every probe as columns
for n in range(n_exp): # experience
    for r in range(n_raf): # mesh
        folder_run = f"{config_exp}_{carac_essai['SetNumber'][n]}_raf_{title_raf[r]}"
        name = folder_run
        for l in range(len(tab_pos)): # probe
            sim[n][r][l] = pd.DataFrame()
            sim[n][r][l]['r+']= np.linspace(0,1, 100)
            sim[n][r][l]['vxl']=np.array(loadText(f"{folder_run}/PAR_{name}_VITESSE_EAU{tab_pos[l]}.son"))[1::3,-1]
            sim[n][r][l]['vyl']=np.array(loadText(f"{folder_run}/PAR_{name}_VITESSE_EAU{tab_pos[l]}.son"))[2::3,-1]
            sim[n][r][l]['vzl']=np.array(loadText(f"{folder_run}/PAR_{name}_VITESSE_EAU{tab_pos[l]}.son"))[3::3,-1]
            sim[n][r][l]['vxg']=np.array(loadText(f"{folder_run}/PAR_{name}_VITESSE_AIR{tab_pos[l]}.son"))[1::3,-1]
            sim[n][r][l]['vyg']=np.array(loadText(f"{folder_run}/PAR_{name}_VITESSE_AIR{tab_pos[l]}.son"))[2::3,-1]
            sim[n][r][l]['vzg']=np.array(loadText(f"{folder_run}/PAR_{name}_VITESSE_AIR{tab_pos[l]}.son"))[3::3,-1]
            sim[n][r][l]['k']  =np.array(loadText(f"{folder_run}/PAR_{name}_K{tab_pos[l]}.son"))[1::,-1]
            sim[n][r][l]['alp']=np.array(loadText(f"{folder_run}/PAR_{name}_ALPHA_AIR{tab_pos[l]}.son"))[1::,-1]
            sim[n][r][l]['DB'] =1000*np.array(loadText(f"{folder_run}/PAR_{name}_DIAMETRE{tab_pos[l]}.son"))[2::2,-1]
            sim[n][r][l]['dp'] =np.array(loadText(f"{folder_run}/PAR_{name}_DP{tab_pos[l]}.son"))[1::,-1]

Lisser les profiles en ne conservant qu'une seule valeur par cellule de maillage

In [ ]:
# # smooth profiles for esthetic reasons : get values in each cell
# get index of each cell
tab_i = [ [ [ [] for l in range(len(tab_pos)) ] for r in range(n_raf) ] for n in range(n_exp) ]
for n in range(n_exp): # experience
    for r in range(n_raf): # mesh
        for l in range(len(tab_pos)): # probe
            i0 = 0
            i = i0+1

            while (i<1000) and (np.abs(sim[n][r][l]['dp'][i]) > 1.e-10) :
                while (np.abs(sim[n][r][l]['dp'][i]-sim[n][r][l]['dp'][i0])<1.e-10) :
                    i+=1
                tab_i[n][r][l] += [i0 + (i-i0)//2]
                i0 = i
# get smooth profiles to store in "simbo"
simbo    = [ [ [ -1 for l in range(len(tab_pos)) ] for r in range(n_raf) ] for n in range(n_exp) ]
index    = ["r+",  "alp", "vzl", "vzg", "k", 'DB']

for n in range(n_exp): # experience
    for r in range(n_raf): # mesh
        for l in range(len(tab_pos)): # probe
            simbo[n][r][l] = pd.DataFrame()
            for ind in index :
                tab_loc = []
                for i in tab_i[n][r][l] :
                    tab_loc += [sim[n][r][l][ind][i]]
                simbo[n][r][l][ind] = np.array(tab_loc)

Récupérer les données expérimentales dans une liste

In [ ]:
# # recover test results
Exp = [ -1 for n in range(n_exp) ]
ExpTurb = [ -1 for n in range(n_exp) ]

for n in range(n_exp): # experience
    Exp_file = f"{build}/../donnees/HIBIKI/Set{n+1}/zD=53.5/exp.csv"
    df = pd.read_csv(Exp_file, sep=",")
    Exp[n] = df

Tracé des différents profils ainsi que les données expérimentales correspondantes

In [ ]:
fig = plt.figure(figsize = (28, 14))
axs = fig.subplots(6, 4 )

# mesh
raf = 1
# probe
l = 0
# marker size for experimental data
ms = 3
# couleurs plot
tab_color = ["saddlebrown", "red",  "orange", "chartreuse", "forestgreen", "lightseagreen", 'cyan', "royalblue",  "indigo", 'purple', 'magenta']
# # many experiments can be grouped using the liquid flow rate J_L
# index corresponding to each group
flow = [[i for i in range(4)],
       [i for i in range(4, 8)],
       [i for i in range(8, 12)],
       [i for i in range(12,16)]]
# Label used in the plot for each group
label_line = [r"Hibiki $J_L = .491$ (ms$^{-1}$)",
               r"Hibiki $J_L = .986$ (ms$^{-1}$)",
               r"Hibiki $J_L = 2.01$ (ms$^{-1}$)",
               r"Hibiki $J_L = 5.$ (ms$^{-1}$)"]
# axis limiters
lim_alp = [.4,.4,.5,.5]
lim_u = [1.2,2.0,4.0,11.]
lim_u_min = [0.2,.7,1.5,3.]
lim_u_prime = [.3,.3,0.3,.6]
lim_d = [5,5,6,5]

# # plot results
for i in range(len(flow)): # values of J_L
    for n in flow[i]:      # experiments with the same J_L
        # Simulation results
        axs[0,i].plot([-1, -2],[-1, -1], "o-", label = f"{carac_essai['SetNumber'][n]}", color=tab_color[n-flow[i][0]], markerfacecolor = "White")
        axs[1,i].plot(simbo[n][raf-1][l]['r+'], simbo[n][raf-1][l]['alp'], color=tab_color[n-flow[i][0]]) # alpha(r/R) : void fraction
        axs[2,i].plot(simbo[n][raf-1][l]['r+'], simbo[n][raf-1][l]['DB'], color=tab_color[n-flow[i][0]])  # d_B(r/R)   : bubble diameter
        axs[3,i].plot(simbo[n][raf-1][l]['r+'], simbo[n][raf-1][l]['vzl'], color=tab_color[n-flow[i][0]]) # U_L(r/R)   : mean liquid velocity
        axs[4,i].plot(simbo[n][raf-1][l]['r+'], simbo[n][raf-1][l]['vzg'], color=tab_color[n-flow[i][0]]) # U_G(r/R)   : mean air velocity
        axs[5,i].plot(simbo[n][raf-1][l]['r+'], np.sqrt(simbo[n][raf-1][l]['k']), color=tab_color[n-flow[i][0]]) # u'_L(r/R) : liquid velocity fluctuations (computed as sqrt(k))
        
        # Experimental points
        axs[1,i].plot(Exp[n]['r/R'],  Exp[n]['alpha'], "o", markersize = ms, color=tab_color[n-flow[i][0]], markerfacecolor = "White")
        axs[2,i].plot(Exp[n]['r/R'],  Exp[n]['Dav'], "o", markersize = ms, color=tab_color[n-flow[i][0]], markerfacecolor = "White")
        axs[3,i].plot(Exp[n]['r/R'],  Exp[n]['u_f'], "o", markersize = ms, color=tab_color[n-flow[i][0]], markerfacecolor = "White")
        axs[4,i].plot(Exp[n]['r/R'],  Exp[n]['u_g'], "o", markersize = ms, color=tab_color[n-flow[i][0]], markerfacecolor = "White")
        axs[5,i].plot(Exp[n]['r/R'],  Exp[n]['up'], "o", markersize = ms, color=tab_color[n-flow[i][0]], markerfacecolor = "White")
        
        # Titles
        axs[0,i].text(0,0, label_line[i], ha = "center", va = "center", fontsize = 14)
        axs[0,i].legend(loc = 'lower center', fontsize = 10, ncol=2)
        axs[0,i].set_xlim(-.5,.5)
        axs[0,i].set_ylim(-.7,.3)
        axs[0,i].axis('off')
        
        # plot labels
        axs[1,i].set_ylabel(r"$\alpha$")
        axs[2,i].set_ylabel(r"$d_B$ (mm)")
        axs[3,i].set_ylabel(r"$U_L$ (ms$^{-1}$)")
        axs[4,i].set_ylabel(r"$U_G$ (ms$^{-1}$)")
        axs[5,i].set_ylabel(r"$k$ (m$^{2}$s$^{-2}$)")
        axs[5,i].set_xlabel(r"$r/R$")
        
        # axis limiters
        axs[1,i].set_ylim(0, lim_alp[i])
        axs[2,i].set_ylim(0, lim_d[i])
        axs[3,i].set_ylim(lim_u_min[i], lim_u[i])
        axs[4,i].set_ylim(lim_u_min[i], lim_u[i])
        axs[5,i].set_ylim(0, lim_u_prime[i])
fig.suptitle("Comparaison des résultats de simulation aux données expérimentales de Hibiki et Ishii", fontsize = 26)
fig.tight_layout()
plt.savefig("results.pdf")